In [1]:
from datasets import concatenate_datasets, load_dataset
from datasets import Dataset, DatasetDict
import pandas as pd
import numpy as np
import torch
import os
import ast
import spacy
# from tqdm import tqdm
import time
# pd.set_option('display.max_colwidth', None)

# Read ABSA-processed Reviews

In [2]:
df = pd.read_pickle("../../data/yelp/reviews_absa_processed.pkl")
df.columns = [col.replace('prompt_', '').replace('aspect', 'aspects').replace('sentiment', 'sentiments') for col in df.columns]
df = df[df['sentiments'] != 'neutral']
df.drop_duplicates(subset=['category', 'review_id', 'sentences']).shape

(23540, 17)

In [3]:
df

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,business_name,categories,categories_list,category,sentences,aspects,sentiments,aspects_lemm
0,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,I love driving on the Benjamin Franklin Bridge...,1352223350000,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmar...","[Automotive, Local Flavor, Active Life, Landma...",Automotive,I love driving on the Benjamin Franklin Bridge.,driving,positive,drive
2,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,I love driving on the Benjamin Franklin Bridge...,1352223350000,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmar...","[Automotive, Local Flavor, Active Life, Landma...",Automotive,The view from the bridge is amazing.,view,positive,view
3,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,I love driving on the Benjamin Franklin Bridge...,1352223350000,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmar...","[Automotive, Local Flavor, Active Life, Landma...",Automotive,I love the waterfront PA/NJ scenery.,scenery,positive,scenery
5,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,I love driving on the Benjamin Franklin Bridge...,1352223350000,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmar...","[Automotive, Local Flavor, Active Life, Landma...",Automotive,The EZ pass makes getting through the tolls a ...,EZ pass,positive,ez pass
6,WWdE3rpUOAuajjJ7F3RQrQ,-NbeVN5tnwdyYAvdNkKMjw,LUXRw-mr9emGL2gw4otvVA,4.0,2,1,2,I love driving on the Benjamin Franklin Bridge...,1352223350000,Benjamin Franklin Bridge,"Automotive, Local Flavor, Active Life, Landmar...","[Automotive, Local Flavor, Active Life, Landma...",Automotive,If you are riding across the Benjamin Franklin...,view,positive,view
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43796,6bN3ZyICqIQupo8OJnnseg,MqpLPBllX_3GJTarGZYMxw,XnNk4Ka8HBiSKh2dEHEqvA,3.0,2,0,0,This place is hidden inside an establishment c...,1557112468000,Mandarin Heights,"Nightlife, Arts & Entertainment, Cocktail Bars...","[Nightlife, Arts & Entertainment, Cocktail Bar...",Arts & Entertainment,"\n\nWhen I was seated however, two very lovely...",complimentary mint juleps,positive,complimentary mint juleps
43797,6bN3ZyICqIQupo8OJnnseg,MqpLPBllX_3GJTarGZYMxw,XnNk4Ka8HBiSKh2dEHEqvA,3.0,2,0,0,This place is hidden inside an establishment c...,1557112468000,Mandarin Heights,"Nightlife, Arts & Entertainment, Cocktail Bars...","[Nightlife, Arts & Entertainment, Cocktail Bar...",Arts & Entertainment,"One of introduced herself by saying ""wow we've...",welcome,positive,welcome
43800,6bN3ZyICqIQupo8OJnnseg,MqpLPBllX_3GJTarGZYMxw,XnNk4Ka8HBiSKh2dEHEqvA,3.0,2,0,0,This place is hidden inside an establishment c...,1557112468000,Mandarin Heights,"Nightlife, Arts & Entertainment, Cocktail Bars...","[Nightlife, Arts & Entertainment, Cocktail Bar...",Arts & Entertainment,Not strong enough to be worth $10 though.,strength,negative,strength
43801,6bN3ZyICqIQupo8OJnnseg,MqpLPBllX_3GJTarGZYMxw,XnNk4Ka8HBiSKh2dEHEqvA,3.0,2,0,0,This place is hidden inside an establishment c...,1557112468000,Mandarin Heights,"Nightlife, Arts & Entertainment, Cocktail Bars...","[Nightlife, Arts & Entertainment, Cocktail Bar...",Arts & Entertainment,"\n\nI came, I saw, I drank, I left...likely ne...",drinks,negative,drink


# Preprocessing

In [4]:
col_agg = {col: lambda x: x.iloc[0] for col in df.columns if col not in ['review_id', 'business_id', 'sentences',
                                                                         'aspect', 'sentiment' ,'aspect_lemm']}
sent_list_agg = {col: lambda x: x.tolist() for col in df.columns if col in ['aspects', 'sentiments' ,'aspects_lemm']}
col_agg.update(sent_list_agg)

In [5]:
# Aggregating
df = df.groupby(['review_id', 'business_id', 'sentences'], sort=False, as_index=False).agg(col_agg).reset_index(drop=True)

In [6]:
# Indexing
df = df.groupby(['review_id', 'business_id'], 
           sort=False, as_index=False).apply(lambda grp: grp.reset_index(drop=True)).reset_index()
df['id'] = df['review_id'].astype(str) + "######" + df['level_1'].astype(str)

In [7]:
df_scored = df

In [8]:
df_scored['num_of_aspects'] = df_scored['aspects_lemm'].apply(lambda x: len(x))

# Aspect Sentiment Clustering

In [9]:
import spacy
nlp = spacy.load('en_core_web_lg')

def cal_spacy_similarity(text1, text2):
    doc1 = nlp(text1)
    doc2 = nlp(text2)
    return doc1.similarity(doc2)

Merge to the cluster with the highest score

In [10]:
def deduplicate(inputs):
    """ Group similar aspect terms of a business in a greedy fashion."""
    # Deduplication
    buss_aspects = inputs[0]
    sent_buss_df = inputs[1]
    
    filtered = []
    for aspect in buss_aspects:
        find_merge = False
        
        similarity_to_other_clusters = []
        # Get best cluster
        for aspects_cluster in filtered:
            average_cosine = average_similarity_to_cluster(aspect, aspects_cluster, sent_buss_df)
            similarity_to_other_clusters += [average_cosine]
        
        sorted_cluster_indices = np.argsort(similarity_to_other_clusters)[::-1]
        
        if len(sorted_cluster_indices) > 0:
            optimal_cluster_index = sorted_cluster_indices[0]
            if similarity_to_other_clusters[optimal_cluster_index] >= threshold:             
                aspects_other = filtered[optimal_cluster_index]
                aspects_other.append(aspect)
                find_merge = True
                
        if not find_merge:
            filtered.append([aspect])

    aspect_clusters_df = pd.DataFrame()
    aspect_clusters_df['aspects_lemm'] = filtered
    aspect_clusters_df = aspect_clusters_df.reset_index().explode(['aspects_lemm']).rename(columns={'index': 'cluster_id'})
    
    return filtered, sent_buss_df.merge(aspect_clusters_df, on=['aspects_lemm'])

In [11]:
import statistics
def average_similarity_to_cluster(kp, kps_other, sent_buss_df):
    """ Calculate average cosine similarity of an AK to a cluster """
    total_similarity = []
    for kp_other in kps_other:
        total_similarity += [calculate_similarity(kp, kp_other, sent_buss_df)]
        
    return statistics.mean(total_similarity)

In [12]:
def calculate_similarity(text1, text2, sent_buss_df):
    """ Determine if two extractions are the same or not
    Args:
        other (Extraction object)
    Returns:
        True or False
    Rule:
        Consider two extractions as the same if their w2v cosine similarity
        is above the specified threshold:
            ext1 == ext2, if cosine(ext1.emb, ext2.emb) >= threshold
    """
    similarity = cal_spacy_similarity(text1, text2)
    return similarity

Apply

In [13]:
# The merging threshold
threshold = 0.55

In [14]:
sent_df = df_scored.explode(['aspects', 'sentiments' ,'aspects_lemm'])

In [15]:
col_agg = {col: lambda x: x.iloc[0] for col in df.columns if col in ['business_name', 'business_id', 'categories', 'categories_list', 'category']}
sent_list_agg = {col: lambda x: x.tolist() for col in df.columns if col not in ['cluster_id', 'business_name', 'business_id', 'categories', 'categories_list', 'category']}
col_agg.update(sent_list_agg)

In [16]:
from tqdm.contrib.concurrent import process_map  # or thread_map
num_workers = 5

In [17]:
inputs = []
for category in sorted(df_scored['category'].unique()):
    for business_id in sorted(df_scored[df_scored['category'] == category]['business_id'].unique()):
        for sentiment in ['positive', 'negative']:
            sent_buss_df = sent_df[(sent_df['business_id'] == business_id) & (sent_df['sentiments'] == sentiment)]
            sent_buss_df = sent_buss_df[sent_buss_df.apply(lambda row: row['aspects'].lower() in row['sentences'].lower(), axis=1)]

            # Sort aspects by their occurrences in the particular business
            sorted_aspects_index = sent_buss_df['aspects_lemm'].value_counts()
            buss_aspects = sorted_aspects_index.index.tolist()
            
            inputs += [(buss_aspects, sent_buss_df)]

In [18]:
start_time = time.time()
clusters_info = process_map(deduplicate, inputs, max_workers=num_workers)
print("TIME ELAPSED", time.time() - start_time)

  0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipykernel_4059/74359008.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc1.similarity(doc2)
/tmp/ipykernel_4059/74359008.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc1.similarity(doc2)
/tmp/ipykernel_4059/74359008.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc1.similarity(doc2)
/tmp/ipykernel_4059/74359008.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc1.similarity(doc2)
/tmp/ipykernel_4059/74359008.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc1.similarity(doc2)
/tmp/ipykernel_4059/74359008.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc1.similarity(doc2)
/tmp/ipykernel_4059/74359008.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc1.similarity(doc2)
/tmp/ipykernel_4059/74359008.py:7: UserWarning: 

/tmp/ipykernel_4059/74359008.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc1.similarity(doc2)
/tmp/ipykernel_4059/74359008.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc1.similarity(doc2)
/tmp/ipykernel_4059/74359008.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc1.similarity(doc2)


TIME ELAPSED 720.6131482124329


In [19]:
dfs = []

for business_sentiment_cluster_info in clusters_info:
    sent_buss_clustered_df = business_sentiment_cluster_info[1]
    
    # Number of sentences in a cluster must be > the number of aspects"
    sent_buss_clustered_df = sent_buss_clustered_df.groupby(['cluster_id']).filter(lambda grp: len(grp) > len(grp['aspects_lemm'].unique()))

    # Get the final clustered df of comments by aspects
    aspect_clusters_df = sent_buss_clustered_df.groupby(['cluster_id']).agg(col_agg)

    aspect_clusters_df['cluster_sentiment'] = aspect_clusters_df['sentiments'].iloc[0][0]

    dfs += [aspect_clusters_df]

In [20]:
summ_df = pd.concat(dfs)
summ_df.to_pickle("../../data/yelp/aspect_sentiment_clusters.pkl")

In [21]:
summ_df

,business_id,business_name,categories,categories_list,category,level_0,level_1,review_id,sentences,user_id,...,useful,funny,cool,date,aspects,sentiments,aspects_lemm,id,num_of_aspects,cluster_sentiment
cluster_id,,,,,,,,,,,,,,,,,,,,,
0,DKFU7w82t-X2WyF0t-qcMA,Engine 8 Urban Winery,"Arts & Entertainment, Wineries, Food","[Arts & Entertainment, Wineries, Food]",Arts & Entertainment,"[4581, 4584, 4584, 4590, 4600, 4600, 4603, 460...","[2, 4, 5, 3, 0, 1, 4, 0, 5, 1, 0, 2, 0, 3, 2, ...","[eSKZ8ItJq_O6q0kATnk3mA, R6MlRNzxt-chAimMNjlf_...","[ Friendly staff, reasonably priced wine by th...","[XrZUSyEhjyNkjc05AUdYoA, kIADx0pgrTJNHUERMDOW0...",...,"[0, 2, 2, 0, 0, 0, 1, 0, 1, 2, 0, 2, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 4, 4, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1557596199000, 1556549106000, 1556549106000, ...","[wine, wine, wine, wine, wines, wine, wine, wi...","[positive, positive, positive, positive, posit...","[wine, wine, wine, wine, wine, wine, wine, win...","[eSKZ8ItJq_O6q0kATnk3mA######2, R6MlRNzxt-chAi...","[2, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, ...",positive
1,DKFU7w82t-X2WyF0t-qcMA,Engine 8 Urban Winery,"Arts & Entertainment, Wineries, Food","[Arts & Entertainment, Wineries, Food]",Arts & Entertainment,"[4588, 4639, 4763, 4827, 4590, 4682, 4748, 474...","[2, 2, 0, 6, 5, 1, 0, 0, 3, 0, 2, 2, 1, 0, 3, ...","[FOC_6X5pxndhP6x0pl5y9A, Sb25U-En0D-hzV_zhGtwW...",[ I'm glad to have this new business in Sparks...,"[0QbByQUNJB7ByoxJe1klbQ, _JzvB98ppovharKXQ0B0T...",...,"[3, 1, 4, 4, 0, 0, 0, 1, 0, 0, 0, 0, 1, 2, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3, 3, 2, 3, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, ...","[1556683450000, 1586903899000, 1558542870000, ...","[business, business, business, business, custo...","[positive, positive, positive, positive, posit...","[business, business, business, business, custo...","[FOC_6X5pxndhP6x0pl5y9A######2, Sb25U-En0D-hzV...","[1, 1, 1, 2, 2, 2, 3, 1, 1, 3, 1, 1, 1, 3, 2, ...",positive
2,DKFU7w82t-X2WyF0t-qcMA,Engine 8 Urban Winery,"Arts & Entertainment, Wineries, Food","[Arts & Entertainment, Wineries, Food]",Arts & Entertainment,"[4593, 4620, 4650, 4667, 4669, 4673, 4681, 469...","[2, 1, 1, 3, 0, 7, 1, 0, 0, 1, 1, 2, 0, 4, 1, ...","[RkVZJE6ZJ9kGc4leeULlmg, nMcIT_Ssh1B4Ozm7onAZ8...","[\nSurprisingly good snacks, too--limited choi...","[q3O2Sgb_WypIFTZoQ9VZyg, 6fVQakoLDzgafQIQSXNvT...",...,"[1, 0, 1, 1, 2, 1, 2, 0, 0, 0, 2, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 1, 1, 0, 1, 2, 0, 0, 0, 2, 0, 0, 0, 0, ...","[1563499273000, 1576114162000, 1584753163000, ...","[snacks, pizzas, pizza, pizza, pizzas, pizza, ...","[positive, positive, positive, positive, posit...","[snack, pizza, pizza, pizza, pizza, pizza, piz...","[RkVZJE6ZJ9kGc4leeULlmg######2, nMcIT_Ssh1B4Oz...","[2, 2, 2, 1, 3, 1, 4, 2, 2, 1, 3, 3, 2, 2, 5, ...",positive
3,DKFU7w82t-X2WyF0t-qcMA,Engine 8 Urban Winery,"Arts & Entertainment, Wineries, Food","[Arts & Entertainment, Wineries, Food]",Arts & Entertainment,"[4581, 4605, 4612, 4620, 4638, 4639, 4642, 467...","[2, 1, 0, 2, 3, 5, 1, 1, 0, 6, 1, 5, 0, 1, 3, 5]","[eSKZ8ItJq_O6q0kATnk3mA, i12Y4WBmXU_Yr_N53ulYG...","[ Friendly staff, reasonably priced wine by th...","[XrZUSyEhjyNkjc05AUdYoA, N1O4vUIhl97PzwfjR74DW...",...,"[0, 0, 2, 0, 1, 1, 5, 1, 0, 1, 4, 2, 0, 0, 4, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1, 3, 6, 1, 0, 0, 1, 1, 0, 0, 2, 0]","[1557596199000, 1559440959000, 1556486083000, ...","[staff, staff, staff, staff, staff, staff, sta...","[positive, positive, positive, positive, posit...","[staff, staff, staff, staff, staff, staff, sta...","[eSKZ8ItJq_O6q0kATnk3mA######2, i12Y4WBmXU_Yr_...","[2, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 2, 3, 4, 1, 2]",positive
4,DKFU7w82t-X2WyF0t-qcMA,Engine 8 Urban Winery,"Arts & Entertainment, Wineries, Food","[Arts & Entertainment, Wineries, Food]",Arts & Entertainment,"[4591, 4644, 4653, 4667, 4675, 4682, 4683, 470...","[3, 0, 1, 2, 1, 